# Benchmark values for the 2h term

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import colossus
from colossus.cosmology import cosmology
from colossus.halo import profile_nfw
from colossus.halo import profile_outer

In [ ]:
# NumCosmo
try:
    import gi

    gi.require_version("NumCosmo", "1.0")
    gi.require_version("NumCosmoMath", "1.0")
except:
    pass

from gi.repository import GObject
from gi.repository import NumCosmo as Nc
from gi.repository import NumCosmoMath as Ncm

In [ ]:
import os

os.environ["CLMM_MODELING_BACKEND"] = "ccl"

In [ ]:
import clmm
from clmm import Cosmology as clmm_cosmo

In [ ]:
cosmo_Ob0 = 0.049
cosmo_Oc0 = 0.262
cosmo_H0 = 67.66

mycosmo = {
    "flat": True,
    "H0": cosmo_H0,
    "Om0": cosmo_Ob0 + cosmo_Oc0,
    "Ob0": cosmo_Ob0,
    "sigma8": 0.8,
    "ns": 0.96,
    "Tcmb0": 2.7255,
}
cosmo_col = cosmology.setCosmology("mycosmo", mycosmo)
print(cosmo_col)

Omega_c_col = cosmo_col.Om0 - cosmo_col.Ob0
print(Omega_c_col)
Omega_b_col = cosmo_col.Ob0
h_col = cosmo_col.H0 / 100.0
n_s_col = cosmo_col.ns

# NumCosmo Cosmology
Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

cosmo = Nc.HICosmo.new_from_name(Nc.HICosmo, "NcHICosmoDEXcdm{'massnu-length':<1>}")
cosmo.omega_x2omega_k()
cosmo.param_set_by_name("H0", cosmo_H0)
cosmo.param_set_by_name("Omegak", 0.0)
cosmo.param_set_by_name("w", -1)
cosmo.param_set_by_name("Omegab", cosmo_Ob0)
cosmo.param_set_by_name("Omegac", cosmo_Oc0)
cosmo.param_set_by_name("ENnu", cosmo_col.Neff)
cosmo.param_set_by_name("Tgamma0", cosmo_col.Tcmb0)
cosmo.param_set_by_name("massnu_0", 0.06)
ENnu = 3.046 - 3.0 * cosmo.E2Press_mnu(1.0e10) / (
    cosmo.E2Omega_g(1.0e10) * (7.0 / 8.0 * (4.0 / 11.0) ** (4.0 / 3.0))
)
cosmo.param_set_by_name("ENnu", ENnu)

reion = Nc.HIReionCamb.new()
cosmo.add_submodel(reion)
prim = Nc.HIPrimPowerLaw.new()
cosmo.add_submodel(prim)
cosmo.prim.props.n_SA = cosmo_col.ns
ps = Nc.PowspecMLTransfer.new(Nc.TransferFuncEH.new())
psf = Ncm.PowspecFilter.new(ps, Ncm.PowspecFilterType.TOPHAT)
old_amplitude = np.exp(cosmo.prim.props.ln10e10ASA)
cosmo.prim.props.ln10e10ASA = np.log((cosmo_col.sigma8 / cosmo.sigma8(psf)) ** 2 * old_amplitude)
ps.prepare(cosmo)

# CLMM cosmology
cosmo_clmm = clmm_cosmo(H0=cosmo_col.H0, Omega_dm0=Omega_c_col, Omega_b0=Omega_b_col, Omega_k0=0.0)

## First, look at P(K)

In [ ]:
z = 1.0

In [ ]:
k_vals = np.logspace(-5, 3, 1000)  # Mpc
pk_col = cosmo_col.matterPowerSpectrum(k_vals / cosmo_col.h, z=z, model="eisenstein98")

In [ ]:
pk_clmm = cosmo_clmm.eval_linear_matter_powerspectrum(k_vals, z)

In [ ]:
pk_nc = []
for k in k_vals:
    pk_nc.append(ps.eval(cosmo, z, k))

In [ ]:
plt.loglog(k_vals, pk_col / (cosmo_col.h) ** 3.0)
plt.loglog(k_vals, pk_clmm)
plt.loglog(k_vals, pk_nc)

In [ ]:
plt.loglog(k_vals, np.abs(pk_clmm / (pk_col / (cosmo_col.h) ** 3.0) - 1.0), label="CLMM/colossus")
plt.loglog(k_vals, np.abs(np.array(pk_clmm) / np.array(pk_nc) - 1.0), label="CLMM/NC")
plt.loglog(
    k_vals,
    np.abs(np.array(pk_nc) / np.array(pk_col / (cosmo_col.h) ** 3.0) - 1.0),
    label="NC/colossus",
)
plt.legend()

In [ ]:
np.savetxt("matter_power_spectrum.txt", np.array([k_vals, pk_nc]).T, header="k  P_of_k")

In [ ]:
config_dict_ps = {
    "cosmo_Ob0": cosmo.Omega_b0(),
    "cosmo_H0": cosmo.H0(),
    "cosmo_Om0": cosmo.Omega_m0(),
    "cosmo_0c0": cosmo.Omega_c0(),
    "z": z,
}

config_dict_ps

In [ ]:
import json

with open("config_ps_benchmarks.json", "w") as fp:
    json.dump(config_dict_ps, fp)

# Now, look at the 2h term with colossus

In [ ]:
M = 1.0e15  # Msun/h
c = 4.0
z = 0.1

In [ ]:
r_3d = np.logspace(-2, 2, 1000)  # Mpc/h
r_proj = np.logspace(-2, 2, 1000)  # Mpc/h

## Define a NFW-only and NFW+2h profile

In [ ]:
p_nfw0 = profile_nfw.NFWProfile(M=M, c=c, z=z, mdef="200c")

outer_term_xi = profile_outer.OuterTermCorrelationFunction(z=z, bias=1.0)
p_nfw_2h = profile_nfw.NFWProfile(M=M, c=c, z=z, mdef="200c", outer_terms=[outer_term_xi])

## Check 3d density

In [ ]:
rho_nfw0 = p_nfw0.density(r_3d * 1.0e3)
rho_nfw_2h = p_nfw_2h.density(r_3d * 1.0e3)
rho_nfw_outer = p_nfw_2h.densityOuter(r_3d * 1.0e3)

In [ ]:
plt.loglog(r_3d, rho_nfw0, label="NFW only")
plt.loglog(r_3d, rho_nfw_outer, label="2h only")
plt.loglog(r_3d, rho_nfw_2h, label="NFW + 2h")
plt.legend()
plt.xlabel("r [Mpc/h]")

## Check surface density

### Check that the inner profile does correspond to NFW-only

In [ ]:
Sigma_nfw0 = p_nfw0.surfaceDensity(r_proj * 1.0e3)
Sigma_nfw_2h_inner = p_nfw_2h.surfaceDensityInner(r_proj * 1.0e3)

In [ ]:
plt.loglog(r_proj, Sigma_nfw_2h_inner / Sigma_nfw0)
plt.xlabel("R [Mpc/h]")

### Compute the 2h term component only (outer profile)
--> get an error w/wo interpolation. Complains about negative densities, but these look OK from above

#### With interpolation

In [ ]:
Sigma_nfw_2h = p_nfw_2h.surfaceDensityOuter(r_proj * 1.0e3, interpolate=True)

#### Without interpolation

In [ ]:
Sigma_nfw_2h = p_nfw_2h.surfaceDensityOuter(r_proj * 1.0e3, interpolate=False)